In [1]:
import pickle
import pandas as pd
import numpy as np
import itertools
import re
import string
import math
import csv
import json
import scipy.spatial as spac

In [5]:
def flatten(array, offset=0):
    _v = []
    for i in array:
        for _j ,j in enumerate(i):
            if(_j>=offset):
                _v.append(j)
    return np.array(_v)

In [6]:
def unpickle(path):
    return pickle.load(open(path, 'rb'))

courses = unpickle("./yaleocw-corpus/data/courses.pickle")
sessions = unpickle("./yaleocw-corpus/data/sessions.pickle")
vectors = unpickle("vectors_doc2vec_sessions.pickle")
preds = unpickle("preds_doc2vec_sessions.pickle")

In [4]:
sessions = sessions.drop(sessions[sessions["Transcriptions"].isna()].index)
sessions = sessions.drop(sessions[sessions["Chapters"].isna()].index)
sessions = sessions.drop([453,454,821,881])

In [5]:
with open('/tmp/vectors.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for _seq in vectors:
        for _chap in _seq:
            tsv_writer.writerow([_i for _i in _chap])
            
with open('/tmp/metadata.tsv', 'wt') as out_file:
    metadata = sessions[["Title", "Chapters", "Cid"]].set_index("Cid").join(courses[["Department", "Course Title"]])
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(metadata.columns)
    for _sid, _seq in metadata.iterrows():
            for _chap in _seq["Chapters"]:
                tsv_writer.writerow([_seq["Title"], _chap[0], _seq["Department"], _seq["Course Title"]])
    

In [6]:
sep = 706
with open('/tmp/pred_vectors.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for _seq in preds:
        for _chap in _seq:
            tsv_writer.writerow([_i for _i in _chap])
            
with open('/tmp/pred_metadata.tsv', 'wt') as out_file:
    metadata = sessions[["Title", "Chapters", "Cid"]].set_index("Cid").join(courses[["Department", "Course Title"]])
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(np.concatenate((metadata.columns, ["Train"])))
    for _sid, _b in enumerate(metadata.iterrows()):
        _cid, _seq = _b
        for _id, _chap in enumerate(_seq["Chapters"]):
            if _id!=0:
                tsv_writer.writerow([_seq["Title"], _chap[0], _seq["Department"], _seq["Course Title"], ("Train", "Test")[_sid<=sep]])

In [7]:
sep = 706
with open('/tmp/all_vectors.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for _seq in vectors:
        for _chap in _seq:
            tsv_writer.writerow([_i for _i in _chap])
    for _seq in preds:
        for _chap in _seq:
            tsv_writer.writerow([_i for _i in _chap])        
            
with open('/tmp/all_metadata.tsv', 'wt') as out_file:
    metadata = sessions[["Title", "Chapters", "Cid"]].set_index("Cid").join(courses[["Department", "Course Title"]])
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(np.concatenate((metadata.columns, ["Train/Test", "In/Out"])))
    for _sid, _b in enumerate(metadata.iterrows()):
        _cid, _seq = _b
        for _id, _chap in enumerate(_seq["Chapters"]):
            tsv_writer.writerow([_seq["Title"], _chap[0], _seq["Department"], _seq["Course Title"], ("Train", "Test")[_sid<=sep], "In"])
    for _sid, _b in enumerate(metadata.iterrows()):
        _cid, _seq = _b
        for _id, _chap in enumerate(_seq["Chapters"]):
            if _id!=0:
                tsv_writer.writerow([_seq["Title"], _chap[0], _seq["Department"], _seq["Course Title"], ("Train", "Test")[_sid<=sep], "Out"])